In [ ]:
!pip install luigi # installing the luigi packages

In [ ]:
import requests
import os
import luigi
import json
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
%load_ext sql
# importing the necessary libraries

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
class Extract_data(luigi.Task):

  def output(self):  # output would be the excel file having the dataframe
    return luigi.LocalTarget('extract.csv')

  def run(self):
    API_URL="https://api.openbrewerydb.org/v1/breweries?by_state=missouri" # Using API to get open source data
    data=requests.get(API_URL).json() # Sending an HTTP GET request to the API and parsing the response as JSON
    df=pd.DataFrame(data)
    df.to_csv(self.output().path, index=False) # Writing the DataFrame to a CSV file at the specified output path, excluding the index column

  def complete(self):
     # Checks if the output file exists, indicating that the task has been completed
    return self.output().exists()

class Transform_data(luigi.Task):
  def requires(self):
    return Extract_data() # Specifies that this task depends on the completion of the 'Extract_data' task

  def output(self):
    return luigi.LocalTarget('transform.csv')

  def run(self):
    # Reading the input CSV file into a DataFrame
    df=pd.read_csv(self.input().path)
    df['state']=df["state"].replace("Missouri","MO")
    df["postal_code"]=df["postal_code"].str.slice(stop=5)
    # Writing the transformed DataFrame to the output CSV file, without the index column
    df.to_csv(self.output().path, index=False)

  def complete(self):
     # Checks if the output file exists, indicating that the task has been completed
    return self.output().exists()

class load1(luigi.Task):

  def requires(self):
     # Specifies that this task depends on the successful completion of the 'Transform_data' task
    return Transform_data()

  def output(self):
    return luigi.LocalTarget('my_lite_store.db')


  def run(self):
    df=pd.read_csv(self.input().path)
    # Connecting to the SQLite database at the specified output path
    con=sqlite3.connect(self.output().path)
    # Writing the DataFrame to the 'mo_brewery' table in the SQLite database, replacing the table if it already exists
    df.to_sql(name="mo_brewery",con=con,if_exists='replace')
    # Committing the transaction to ensure changes are saved to the database
    con.commit()
    # Closing the connection to the SQLite database
    con.close()

  def complete(self):
     # Checks if the output file exists, indicating that the task has been completed
    return self.output().exists()

class load2(luigi.Task):
  def requires(self):
     # Specifies that this task depends on the successful completion of the 'Transform_data' task
    return Transform_data()

  def output(self):
    # Specifies the output file path where the task's results will be stored (my_output.csv)
    return luigi.LocalTarget('my_output.csv')

  def run(self):
    df=pd.read_csv(self.input().path)
    df.to_csv(self.output().path,index=False)

# Builds and executes the tasks 'load1' and 'load2' in the correct order, with local scheduling enabled
luigi.build([load1(), load2()], local_scheduler=True)



DEBUG: Checking if load1() is complete
DEBUG:luigi-interface:Checking if load1() is complete
DEBUG: Checking if Transform_data() is complete
DEBUG:luigi-interface:Checking if Transform_data() is complete
INFO: Informed scheduler that task   load1__99914b932b   has status   PENDING
INFO:luigi-interface:Informed scheduler that task   load1__99914b932b   has status   PENDING
DEBUG: Checking if Extract_data() is complete
DEBUG:luigi-interface:Checking if Extract_data() is complete
INFO: Informed scheduler that task   Transform_data__99914b932b   has status   PENDING
INFO:luigi-interface:Informed scheduler that task   Transform_data__99914b932b   has status   PENDING
INFO: Informed scheduler that task   Extract_data__99914b932b   has status   PENDING
INFO:luigi-interface:Informed scheduler that task   Extract_data__99914b932b   has status   PENDING
DEBUG: Checking if load2() is complete
DEBUG:luigi-interface:Checking if load2() is complete
DEBUG: Checking if Transform_data() is complete
DEB

True

In [ ]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%%sql
sqlite:///my_lite_store.db # Connects to the SQLite database 'my_lite_store.db' for running SQL queries in the notebook

In [ ]:
%%sql
select * from mo_brewery limit 10

 * sqlite:///my_lite_store.db
Done.


index,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
0,84337872-75da-4cdd-9f8f-1e1691f21642,21st St Brewers Bar,brewpub,2017 Chouteau Ave,None,None,Saint Louis,Missouri,63103,United States,-90.213781,38.624291,3142416969.0,http://www.21stbrew.com,MO,2017 Chouteau Ave
1,be0ec266-d637-42e4-94ba-4ce21e456054,2nd Shift Brewing Co,brewpub,1601 Sublette Ave,None,None,Saint Louis,Missouri,63110,United States,-90.280526,38.621943,6187910728.0,http://www.2ndshiftbrewing.com,MO,1601 Sublette Ave
2,cb56e27e-d113-4f74-925a-69305a56d082,3 Trails Brewing Co,micro,111 N Main St,None,None,Independence,Missouri,64050,United States,-94.41265,39.122385,8168866256.0,http://www.3trailsbrewing.com,MO,111 N Main St
3,4f70ac94-eca5-4cec-8581-92ec5afdf4f2,3Halves Brewing Co,brewpub,110 E Kansas St,None,None,Liberty,Missouri,64068,United States,-94.4190247,39.2461993,8164296886.0,http://www.3halvesbrewingco.com,MO,110 E Kansas St
4,b72f2ce6-e062-404d-ac31-c1060db2d870,4 By 4 Brewing Company,micro,2811 E Galloway St Ste A,None,None,Springfield,Missouri,65804,United States,-93.239889,37.147986,4178616400.0,http://www.4by4brewingcompany.com,MO,2811 E Galloway St Ste A
5,ac2f41ed-e1e8-4586-aa78-253543db7714,4 Hands Brewing Co,regional,1220 S 8th St,None,None,Saint Louis,Missouri,63104,United States,-90.19762012,38.61526593,3144361559.0,http://www.4handsbrewery.com,MO,1220 S 8th St
6,085c97d4-5a01-4e28-a1c6-683bf773b062,Alma Mader Brewing,micro,2635 Southwest Blvd,None,None,Kansas City,Missouri,64108,United States,-94.599389,39.082495,8169452589.0,http://almamaderbrewing.com,MO,2635 Southwest Blvd
7,0edb8050-2911-4a04-a99a-7927e05cd018,Alpha Brewing Co.,micro,4310 Fyler Ave,None,None,Saint Louis,Missouri,63116,United States,-90.26236838,38.59861817,3146212337.0,http://www.alphabrewingcompany.com,MO,4310 Fyler Ave
8,32dd19c9-33b8-4ed5-b5f0-1fb5f5d8ad02,Amerisports Brew Pub,brewpub,3200 Ameristar Dr,None,None,Kansas City,Missouri,64161,United States,-94.484526,39.150682,8164147435.0,http://www.ameristar.com,MO,3200 Ameristar Dr
9,d1a61260-d377-4e7d-8ce9-8851e30934ae,Anheuser-Busch InBev,large,1 Busch Pl,None,None,Saint Louis,Missouri,63118,United States,-90.2118998,38.5954536,3145772000.0,http://www.anheuser-busch.com,MO,1 Busch Pl
